In [1]:
#codigo de geração dos obj

In [2]:
import math

In [3]:
def salvar_esfera_com_cilindro(nome_arquivo, radius=1.0, stacks=10, sectors=20, altura_cilindro=1.0):
    with open(nome_arquivo, 'w') as f:
        f.write("# Meia Esfera com Cilindro abaixo\n")
        f.write("mtllib none\n")

        vertices = []

        # === MEIA ESFERA ===
        for i in range(stacks + 1):
            stack_angle = math.pi / 2 * (i / stacks)  # de 0 até pi/2
            xy = radius * math.cos(stack_angle)
            z = radius * math.sin(stack_angle)

            for j in range(sectors + 1):
                sector_angle = 2 * math.pi * (j / sectors)
                x = xy * math.cos(sector_angle)
                y = xy * math.sin(sector_angle)

                # Inverte y <-> z
                vertices.append((x, z, y))

                f.write(f"v {x:.6f} {z:.6f} {y:.6f}\n")

        # === CILINDRO ===
        base_index = len(vertices)  # para ajustar índices das faces

        for i in range(2):  # topo e base
            y = 0 if i == 0 else -altura_cilindro
            for j in range(sectors + 1):
                angle = 2 * math.pi * (j / sectors)
                x = radius * math.cos(angle)
                z = radius * math.sin(angle)

                vertices.append((x, y, z))

                f.write(f"v {x:.6f} {y:.6f} {z:.6f}\n")

        # === FACES MEIA ESFERA ===
        for i in range(stacks):
            for j in range(sectors):
                v1 = i * (sectors + 1) + j + 1
                v2 = v1 + sectors + 1
                v3 = v1 + 1
                v4 = v2 + 1

                f.write(f"f {v1} {v2} {v3}\n")
                f.write(f"f {v3} {v2} {v4}\n")

        # === FACES CILINDRO ===
        for j in range(sectors):
            i1 = base_index + j + 1
            i2 = i1 + sectors + 1
            i3 = i1 + 1
            i4 = i2 + 1

            f.write(f"f {i1} {i2} {i3}\n")
            f.write(f"f {i3} {i2} {i4}\n")

    print(f"Arquivo '{nome_arquivo}' gerado com meia esfera + cilindro encaixado!")

# Exemplo de uso:
salvar_esfera_com_cilindro("corpo_fantasma.obj")

Arquivo 'corpo_fantasma.obj' gerado com meia esfera + cilindro encaixado!


In [4]:
def salvar_arvore_com_cone_vt(nome_arquivo, altura_tronco=0.8, raio_tronco=0.2,
                               altura_copa=2.0, raio_copa=0.6, setores=20):
    with open(nome_arquivo, 'w') as f:
        f.write("# Árvore com cone como copa, usando vt e sem vn\n")

        vertices = []

        # === TRONCO (CILINDRO) ===
        for i in range(2):  # base (0) e topo (1)
            y = i * altura_tronco
            for j in range(setores + 1):
                ang = 2 * math.pi * j / setores
                x = raio_tronco * math.cos(ang)
                z = raio_tronco * math.sin(ang)
                vertices.append((x, y, z))
                f.write(f"v {x:.6f} {y:.6f} {z:.6f}\n")

        # === COPA (CONE) ===
        for j in range(setores + 1):
            ang = 2 * math.pi * j / setores
            x = raio_copa * math.cos(ang)
            z = raio_copa * math.sin(ang)
            y = altura_tronco
            vertices.append((x, y, z))
            f.write(f"v {x:.6f} {y:.6f} {z:.6f}\n")

        # topo do cone (pico)
        topo_y = altura_tronco + altura_copa
        vertices.append((0, topo_y, 0))
        f.write(f"v 0.000000 {topo_y:.6f} 0.000000\n")

        # === FACES TRONCO ===
        for j in range(setores):
            v1 = j + 1
            v2 = v1 + setores + 1
            v3 = v1 + 1
            v4 = v2 + 1
            f.write(f"f {v1} {v2} {v3}\n")
            f.write(f"f {v3} {v2} {v4}\n")

        # === FACES COPA ===
        offset = len(vertices) - (setores + 1 + 1)  # início da base da copa
        topo_idx = len(vertices)                   # índice do pico do cone (1-based)
        for j in range(setores):
            base1 = offset + j + 1
            base2 = offset + (j + 1) % setores + 1
            f.write(f"f {base1} {base2} {topo_idx}\n")

    print(f"Arquivo '{nome_arquivo}' gerado com sucesso!")

# Exemplo de uso:
salvar_arvore_com_cone_vt("arvore.obj")

Arquivo 'arvore.obj' gerado com sucesso!
